In [1]:
import pickle
import numpy as np
import pandas as pd

In [2]:
# Cargar el modelo
with open('mejor_modelo_xgboost.pkl', 'rb') as f:
    model = pickle.load(f)
# Cargar los transformers
with open('transformer_scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

with open('transformer_target.pkl', 'rb') as f:
    target = pickle.load(f)

with open('transformer_one.pkl', 'rb') as f:
    one = pickle.load(f)

In [3]:
variables_one = ['MaritalStatus','JobRole','ViajeDeNegocios', 'Departamento','CampoDeEstudio']
data = {
    'Age': 51,
    'ViajeDeNegocios': 'Travel_Rarely',
    'Departamento': 'Sales',
    'DistanciaDesdeCasa': 6,
    'Educación': 2,
    'CampoDeEstudio': 'Life Sciences',
    'Gender': 'Female',
    'JobRole': 'Healthcare Representative',
    'MaritalStatus': 'Married',
    'MonthlyIncome': 131160,
    'IncrementoPorcentajeSalario': 11,
    'NivelDeOpcionesAcciones': 0,
    'VecesDeCapacitaciónAñoPasado': 6,
    'AñosEnLaEmpresa': 1,
    'AñosDesdeÚltimoAscenso': 0,
    'AñosConGerenteActual': 0,
    'CompromisoConElTrabajo': 3,
    'CalificaciónDeDesempeño': 3,
    'SatisfacciónConElEntorno_knn': 3.0,  # Se toma como proxy de 'EnvironmentSatisfaction'
    'SatisfacciónLaboral_knn': 4.0,  # Se toma como proxy de 'JobSatisfaction'
    'NumCompaniesWorked_knn': 1.0,  # Equivalente a 'NumCompaniesWorked'
    'AñosTotalesDeTrabajo_knn': 1.0  # Equivalente a 'TotalWorkingYears'
}


In [4]:
df_pred = pd.DataFrame(data, index = [0])
df_pred.head(2)

,Age,ViajeDeNegocios,Departamento,DistanciaDesdeCasa,Educación,CampoDeEstudio,Gender,JobRole,MaritalStatus,MonthlyIncome,...,VecesDeCapacitaciónAñoPasado,AñosEnLaEmpresa,AñosDesdeÚltimoAscenso,AñosConGerenteActual,CompromisoConElTrabajo,CalificaciónDeDesempeño,SatisfacciónConElEntorno_knn,SatisfacciónLaboral_knn,NumCompaniesWorked_knn,AñosTotalesDeTrabajo_knn
0,51,Travel_Rarely,Sales,6,2,Life Sciences,Female,Healthcare Representative,Married,131160,...,6,1,0,0,3,3,3.0,4.0,1.0,1.0


In [5]:
col_numericas = df_pred.select_dtypes(include = np.number).columns
df_pred[col_numericas] = scaler.transform(df_pred[col_numericas])


In [ ]:
# transformamos las categorias one-hot
df_one = pd.DataFrame(one.transform(df_pred[variables_one]).toarray(), columns=one.get_feature_names_out())
df_pred = pd.concat([df_pred, df_one], axis = 1)
df_pred.drop(columns = variables_one, axis = 1, inplace = True)

,Age,ViajeDeNegocios,Departamento,DistanciaDesdeCasa,Educación,CampoDeEstudio,Gender,JobRole,MaritalStatus,MonthlyIncome,...,ViajeDeNegocios_Travel_Rarely,Departamento_Human Resources,Departamento_Research & Development,Departamento_Sales,CampoDeEstudio_Human Resources,CampoDeEstudio_Life Sciences,CampoDeEstudio_Marketing,CampoDeEstudio_Medical,CampoDeEstudio_Other,CampoDeEstudio_Technical Degree
0,1.552222,Travel_Rarely,Sales,-0.390933,-0.884892,Life Sciences,Female,Healthcare Representative,Married,1.407855,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0


In [7]:
df_pred.columns

Index(['Age', 'ViajeDeNegocios', 'Departamento', 'DistanciaDesdeCasa',
       'Educación', 'CampoDeEstudio', 'Gender', 'JobRole', 'MaritalStatus',
       'MonthlyIncome', 'IncrementoPorcentajeSalario',
       'NivelDeOpcionesAcciones', 'VecesDeCapacitaciónAñoPasado',
       'AñosEnLaEmpresa', 'AñosDesdeÚltimoAscenso', 'AñosConGerenteActual',
       'CompromisoConElTrabajo', 'CalificaciónDeDesempeño',
       'SatisfacciónConElEntorno_knn', 'SatisfacciónLaboral_knn',
       'NumCompaniesWorked_knn', 'AñosTotalesDeTrabajo_knn',
       'MaritalStatus_Divorced', 'MaritalStatus_Married',
       'MaritalStatus_Single', 'JobRole_Healthcare Representative',
       'JobRole_Human Resources', 'JobRole_Laboratory Technician',
       'JobRole_Manager', 'JobRole_Manufacturing Director',
       'JobRole_Research Director', 'JobRole_Research Scientist',
       'JobRole_Sales Executive', 'JobRole_Sales Representative',
       'ViajeDeNegocios_Non-Travel', 'ViajeDeNegocios_Travel_Frequently',
       'Vi

In [8]:
target

TargetEncoder(cols=['Gender', 'JobLevel'])

In [ ]:
 # transformamos las categoricas de target
df_pred = target.transform(df_pred)

ValueError: Unexpected input dimension 46, expected 43